In [4]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np

In [5]:
chemin_bd = r"./bd_ignore/"

In [6]:
df_filtered_newactor2 = pd.read_csv(chemin_bd + 'resultat/df_filtered_newactor.csv')

FileNotFoundError: [Errno 2] No such file or directory: './bd_ignore/resultat/df_filtered_newactor.csv'

In [ ]:
# Encodage des genres

features = ['popularity', 'vote_average', 'vote_count', 'budget', 'revenue', 'runtime']

df_encoded = pd.concat(
    [df_filtered_newactor2[features], pd.get_dummies(df_filtered_newactor2['genres'], prefix='genre')],
    axis=1
)

# Normalisation des données
scaler = MinMaxScaler()  #
X = scaler.fit_transform(df_encoded)

# Modèle Nearest Neighbors
k = 11 # Meilleure valeur de K = 6 d'aprés fig

model = NearestNeighbors(n_neighbors=k, metric='euclidean')
model.fit(X)

# Fonction pour rechercher des films similaires
def films_similaires(film_index):
    distances, indices = model.kneighbors([X[film_index]])
    print("Films similaires :")
    for i, index in enumerate(indices[0]):
        film = df_filtered_newactor2.iloc[index]
        print(f"{i + 1}: {film['title']} (distance: {distances[0][i]:.2f})")
        print(f"   Acteurs: {film['actors']}")

# Exemple d'utilisation
film_index = 0  # Index du film pour lequel vous voulez des recommandations
films_similaires(film_index)


In [ ]:
# Réinitialiser les indices de df_filtered avant de normaliser les données
df_filtered_newactor2 = df_filtered_newactor2.reset_index(drop=True)

# Normalisation des données
scaler = MinMaxScaler()
X = scaler.fit_transform(df_encoded)  # X doit être basé sur df_filtered encodé

In [ ]:
def visualize_all_movie_distances(champion_movies):
    # Préparation des données pour la PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)  # X est votre matrice de données normalisées

    # DataFrame pour les films
    pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
    pca_df['Title'] = df_filtered_newactor2['title'].values  # Ajout des titres
    pca_df['Type'] = 'Movie'

    # Création du graphique
    fig = go.Figure()

    # Ajout des points pour tous les films
    fig.add_trace(go.Scatter(
        x=pca_df['PC1'],
        y=pca_df['PC2'],
        mode='markers',
        marker=dict(color='lightgray', size=8, opacity=0.5),
        name='Movies',
        hovertext=pca_df['Title'],
        hoverinfo='text'
    ))

    # Couleurs pour distinguer les films sélectionnés
    colors = px.colors.qualitative.Set1

    # Vérification des titres valides
    for i, movie_title in enumerate(champion_movies):
        if movie_title not in df_filtered_newactor2['title'].values:
            print(f"Titre non trouvé dans le dataset : {movie_title}")
            continue  # Passer au titre suivant

        # Récupération des caractéristiques du film
        movie_index = df_filtered_newactor2[df_filtered_newactor2['title'] == movie_title].index[0]
        movie_encoded = X[movie_index]  # Index aligné avec df_filtered_newactor2 après reset_index()

        # Transformation PCA
        movie_pca = pca.transform([movie_encoded])

        # Ajout du film sélectionné
        fig.add_trace(go.Scatter(
            x=[movie_pca[0, 0]],
            y=[movie_pca[0, 1]],
            mode='markers+text',
            marker=dict(color=colors[i % len(colors)], size=15, symbol='star'),
            name=movie_title,
            text=[movie_title],
            textposition="top center",
            hoverinfo='text'
        ))

        # Récupération des plus proches voisins
        distances, indices = model.kneighbors([movie_encoded])
        similar_movies = df_filtered_newactor2.iloc[indices[0][1:4]]  # Les 3 plus proches voisins

        # Trouver les coordonnées PCA des films similaires
        similar_pca = pca.transform(X[indices[0][1:4]])

        # Ajout des films similaires avec des lignes les reliant au film sélectionné
        for j in range(len(similar_pca)):
            # Ligne reliant le film sélectionné à son similaire
            fig.add_trace(go.Scatter(
                x=[movie_pca[0, 0], similar_pca[j, 0]],
                y=[movie_pca[0, 1], similar_pca[j, 1]],
                mode='lines',
                line=dict(color=colors[i % len(colors)], width=1, dash='dot'),
                showlegend=False,
                hoverinfo='none'
            ))

            # Point du film similaire
            fig.add_trace(go.Scatter(
                x=[similar_pca[j, 0]],
                y=[similar_pca[j, 1]],
                mode='markers+text',
                marker=dict(color=colors[i % len(colors)], size=10),
                name=f'Similar to {movie_title}',
                text=[similar_movies.iloc[j]['title']],
                textposition="top center",
                showlegend=False,
                hovertext=[f"Similar to {movie_title}: {similar_movies.iloc[j]['title']}"],
                hoverinfo='text'
            ))

    # Mise à jour du layout
    fig.update_layout(
        title='PCA: Selected Movies and Their Similar Movies',
        xaxis_title=f'PC1 (Variance: {pca.explained_variance_ratio_[0]:.2%})',
        yaxis_title=f'PC2 (Variance: {pca.explained_variance_ratio_[1]:.2%})',
        hovermode='closest',
        width=800,
        height=800
    )

    # Afficher le graphique
    fig.show()

    # Afficher l'information sur la variance expliquée totale
    total_var = pca.explained_variance_ratio_.sum()
    print(f"\nTotal variance explained by these 2 components: {total_var:.2%}")

# Utilisation de la fonction
selected_movies = ['Inception', 'Avatar', 'The Dark Knight']  # Exemple de films sélectionnés
visualize_all_movie_distances(selected_movies)


In [ ]:

# Fonction pour évaluer différentes valeurs de k
def evaluate_k(X_encoded, k_range):
    avg_distances = []

    for k in k_range:
        # Modèle Nearest Neighbors
        model = NearestNeighbors(n_neighbors=k, algorithm='auto', metric='euclidean')
        model.fit(X_encoded)

        # Moyenne des distances aux k plus proches voisins
        distances, _ = model.kneighbors(X_encoded)
        avg_distance = distances[:, -1].mean()  # Moyenne de la dernière colonne (k-ème distance)
        avg_distances.append(avg_distance)

    return avg_distances

# Définition de la plage de k à tester
k_range = range(1, 21)  # Test des valeurs de k de 1 à 20

# Évaluation des différentes valeurs de k
X_sample = X[:1000]  # Limiter à un sous-échantillon pour accélérer
avg_distances = evaluate_k(X_sample, k_range)

# Création d'une visualisation pour aider à choisir k
plt.figure(figsize=(6, 5))
plt.plot(k_range, avg_distances, 'bo-')
plt.xlabel('Nombre de voisins (k)')
plt.ylabel('Distance moyenne aux voisins')
plt.title('Distance moyenne en fonction de k')
plt.grid(True)
plt.show()

In [ ]:
def evaluate_k_with_silhouette(X_encoded, k_range):
    avg_distances = []
    silhouette_scores = []

    for k in k_range:
        # Modèle Nearest Neighbors
        model = NearestNeighbors(n_neighbors=k, algorithm='auto', metric='euclidean')
        model.fit(X_encoded)

        # Moyenne des distances aux k plus proches voisins
        distances, _ = model.kneighbors(X_encoded)
        avg_distance = distances[:, -1].mean()
        avg_distances.append(avg_distance)

        # Clustering avec K-Means pour silhouette_score
        if k > 1:
            kmeans = KMeans(n_clusters=k, random_state=42).fit(X_encoded)
            labels = kmeans.labels_
            silhouette = silhouette_score(X_encoded, labels)
            silhouette_scores.append(silhouette)
        else:
            silhouette_scores.append(0)

    return avg_distances, silhouette_scores

# Définition de la plage de k à tester
k_range = range(2, 21)  # K-Means nécessite au moins k=2

# Évaluation des distances moyennes et du score de silhouette
avg_distances, silhouette_scores = evaluate_k_with_silhouette(X_sample, k_range)

# Création d'une visualisation pour aider à choisir k
plt.figure(figsize=(12, 5))

# Premier graphique : Distance moyenne aux voisins
plt.subplot(1, 2, 1)
plt.plot(k_range, avg_distances, 'bo-')
plt.xlabel('Nombre de voisins (k)')
plt.ylabel('Distance moyenne aux voisins')
plt.title('Distance moyenne en fonction de k')
plt.grid(True)

# Second graphique : Score de silhouette
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Score de silhouette')
plt.title('Score de silhouette en fonction de k')
plt.grid(True)

plt.tight_layout()
plt.show()
